# Azure Learning - App Service

In [1]:
az login

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "415d05c4-204b-4df3-b7db-fb5faa4abb73",
    "id": "586f41cd-533d-41bd-9be4-88f5657a9ed4",
    "isDefault": false,
    "managedByTenants": [],
    "name": "Azure Cloud POC",
    "state": "Enabled",
    "tenantId": "415d05c4-204b-4df3-b7db-fb5faa4abb73",
    "user": {
      "name": "Randall.Thornton@cc-racks.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "415d05c4-204b-4df3-b7db-fb5faa4abb73",
    "id": "139ab8b0-8b6c-4574-ba4e-cb7f38419e03",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Visual Studio Professional Subscription",
    "state": "Enabled",
    "tenantId": "415d05c4-204b-4df3-b7db-fb5faa4abb73",
    "user": {
      "name": "Randall.Thornton@cc-racks.com",
      "type": "user"
    }
  }
]


In [2]:
# Set the subscription

az account set --subscription '139ab8b0-8b6c-4574-ba4e-cb7f38419e03'

In [5]:
# Create the resource group
$resourceGroupName = "az-learn-group"
$appServicePlanName = "appServicePlan"
$webAppName = "az-learn-web-app-89"

az group create --name "$resourceGroupName" --location eastus

{
  "id": "/subscriptions/139ab8b0-8b6c-4574-ba4e-cb7f38419e03/resourceGroups/az-learn-group",
  "location": "eastus",
  "managedBy": null,
  "name": "az-learn-group",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


In [4]:
az deployment group create --resource-group "$resourceGroupName" --template-file .\main.bicep --parameters appServicePlanName=$appServicePlanName webAppName=$webAppName

ERROR: Missing input parameters: webAppname


Error: Command failed: SubmitCode: az deployment group create --resource-group "$reso ...